# Model: Predict if the Best nba team will win a given game

Aim of this modle is to predict if the best team is going to win or not the game
The best team is the team with the best win ratio.

## Library import 

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

## Team Names for one season

In [2]:
import os
os.getcwd()

'/Users/yannickflores/Perso Project/foreshadownba/foreshadwownba-data-engineering-pipeline'

In [13]:
def webscrappe_nba_games_data(SEASON):
    
    #------------------------------------------------------
    # Get team reference data 

    team_city_refdata = pd.read_csv('./constants/team_city_refdata.csv', sep = ';')

    #------------------------------------------------------
    # Initialization of the dataframe to fill-in
    games = pd.DataFrame()

    #------------------------------------------------------
    # For Loop Throught all the team abrev for the given season

    for index, row in team_city_refdata.iterrows():
        # URL to scrape
        team = row['team_abrev']
        url = f"https://www.basketball-reference.com/teams/{team}/{SEASON}/gamelog/"

        # collect HTML data and create beautiful soup object:
        # collect HTML data
        html = urlopen(url)
                
        # create beautiful soup object from HTML
        soup = BeautifulSoup(html, "html.parser" )

        rows = soup.findAll('tr')[2:]

        rows_data = [[td.getText() for td in rows[i].findAll('td')]
                            for i in range(len(rows))]

        if len(rows_data) != 0:
            # create the dataframe
            games_tmp = pd.DataFrame(rows_data)
            cols = ["game_nb", "game_date", "extdom", "opp", "results",
                    "pts_tm","pts_opp",
                    "fg_tm", "fga_tm","fg_prct_tm",
                    "3p_tm","3pa_tm", "3p_prct_tm","ft_tm","fta_tm","ft_prct_tm",
                    "orb_tm","trb_tm", "ast_tm","stl_tm","blk_tm" ,"tov_tm","pf_tm",
                    "nc",
                    "fg_opp","fga_opp","fg_prct_opp",
                    "3p_opp", "3pa_opp", "3p_prct_opp", "ft_opp", "fta_opp","ft_prct_opp",
                    "orb_opp", "trb_opp","ast_opp", "stl_opp", "blk_opp","tov_opp", "pf_opp"]

            games_tmp.columns =  cols
            games_tmp = games_tmp.dropna()
            games_tmp['id_season'] = SEASON
            games_tmp['tm'] = team

            games = games.append(games_tmp)

            # print(team)
            # print(games_tmp.shape[0])
            # print(' ')

    games = games[[
        'id_season', 'game_nb', 'game_date', 'extdom', 'tm','opp', 'results', 'pts_tm', 'pts_opp',
        'fg_tm', 'fga_tm', 'fg_prct_tm', '3p_tm', '3pa_tm', '3p_prct_tm',
        'ft_tm', 'fta_tm', 'ft_prct_tm', 'orb_tm', 'trb_tm', 'ast_tm', 'stl_tm',
        'blk_tm', 'tov_tm', 'pf_tm', 'fg_opp', 'fga_opp', 'fg_prct_opp',
        '3p_opp', '3pa_opp', '3p_prct_opp', 'ft_opp', 'fta_opp', 'ft_prct_opp',
        'orb_opp', 'trb_opp', 'ast_opp', 'stl_opp', 'blk_opp', 'tov_opp',
        'pf_opp']]

    return games

In [14]:
test = webscrappe_nba_games_data(2018)
test.head()

,id_season,game_nb,game_date,extdom,tm,opp,results,pts_tm,pts_opp,fg_tm,fga_tm,fg_prct_tm,3p_tm,3pa_tm,3p_prct_tm,ft_tm,fta_tm,ft_prct_tm,orb_tm,trb_tm,ast_tm,stl_tm,blk_tm,tov_tm,pf_tm,fg_opp,fga_opp,fg_prct_opp,3p_opp,3pa_opp,3p_prct_opp,ft_opp,fta_opp,ft_prct_opp,orb_opp,trb_opp,ast_opp,stl_opp,blk_opp,tov_opp,pf_opp
0,2018,1,2017-10-18,@,ATL,DAL,W,117,111,48,94,.511,9,18,.500,12,15,.800,14,50,20,11,6,13,18,38,86,.442,17,45,.378,18,21,.857,9,41,27,5,4,15,19
1,2018,2,2017-10-20,@,ATL,CHO,L,91,109,36,95,.379,7,30,.233,12,14,.857,6,38,19,11,4,13,29,37,86,.430,11,36,.306,24,30,.800,12,57,17,5,4,21,18
2,2018,3,2017-10-22,@,ATL,BRK,L,104,116,32,94,.340,7,27,.259,33,36,.917,16,48,21,6,2,16,25,41,87,.471,11,27,.407,23,34,.676,13,51,27,12,8,18,27
3,2018,4,2017-10-23,@,ATL,MIA,L,93,104,36,76,.474,7,17,.412,14,18,.778,6,41,14,9,1,20,17,39,85,.459,14,37,.378,12,18,.667,10,41,21,12,6,19,19
4,2018,5,2017-10-26,@,ATL,CHI,L,86,91,32,81,.395,11,28,.393,11,13,.846,7,40,22,10,5,10,23,31,86,.360,7,32,.219,22,29,.759,18,62,20,5,4,13,15


In [6]:

def webscrappe_nba_schedule_overtime_data(SEASON):
    
    #------------------------------------------------------
    # Get team reference data 

    team_city_refdata = pd.read_csv('./constants/team_city_refdata.csv', sep = ';')

    #------------------------------------------------------
    # Initialization of the dataframe to fill-in
    schedules = pd.DataFrame()

    #------------------------------------------------------
    # For Loop Throught all the team abrev for the given season

    for index, row in team_city_refdata.iterrows():
        
        # URL to scrape
        team = row['team_abrev']

        # URL to scrape
        url = f"https://www.basketball-reference.com/teams/{team}/{SEASON}_games.html"

        if str(requests.get(url)) != '<Response [404]>':

            # collect HTML data and create beautiful soup object:
            # collect HTML data
            html = urlopen(url)
                    
            # create beautiful soup object from HTML
            soup = BeautifulSoup(html,  "html.parser")

            # use getText()to extract the headers into a list
            titles = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]

            rows = soup.findAll('tr')[1:]

            rows_data = [[td.getText() for td in rows[i].findAll('td')]
                            for i in range(len(rows))]

            if len(rows_data) != 0:
                # create the dataframe
                schedule_tmp = pd.DataFrame(rows_data)

                cols = ['game_date', 'time_start', 'nc1', 'nc2', 'extdom', 'opponent', 'w_l', 'overtime', 'pts_tm', 'pts_opp', 'w_tot', 'l_tot', 'streak_w_l', 'nc3']
                schedule_tmp.columns =  cols

                schedule_tmp = schedule_tmp.dropna()

                schedule_tmp['id_season'] = SEASON
                schedule_tmp['tm'] = team

                schedules = schedules.append(schedule_tmp)

    schedules = schedules[['id_season', 'tm', 'game_date', 'time_start', 'extdom', 'opponent', 'w_l', 'overtime', 'pts_tm', 'pts_opp', 'w_tot', 'l_tot', 'streak_w_l']]
            
    return schedules

In [7]:
test2 = webscrappe_nba_schedule_overtime_data(2001)

test2.head()


,id_season,tm,game_date,time_start,extdom,opponent,w_l,overtime,pts_tm,pts_opp,w_tot,l_tot,streak_w_l
0,2001,ATL,"Tue, Oct 31, 2000",7:30p,,Charlotte Hornets,L,,82,106,0,1,L 1
1,2001,ATL,"Thu, Nov 2, 2000",7:30p,@,New York Knicks,L,,69,94,0,2,L 2
2,2001,ATL,"Sat, Nov 4, 2000",7:30p,,Orlando Magic,L,,104,107,0,3,L 3
3,2001,ATL,"Mon, Nov 6, 2000",10:00p,@,Vancouver Grizzlies,L,,87,97,0,4,L 4
4,2001,ATL,"Tue, Nov 7, 2000",10:00p,@,Portland Trail Blazers,L,,88,97,0,5,L 5


In [33]:
#----------------------------------------------
# Re format date

test2['game_date'] = pd.to_datetime(test2['game_date'])
test2.head()

,id_season,tm,game_date,time_start,extdom,opponent,w_l,overtime,pts_tm,pts_opp,w_tot,l_tot,streak_w_l
0,2018,ATL,2017-10-18,8:30p,@,Dallas Mavericks,W,,117,111,1,0,W 1
1,2018,ATL,2017-10-20,7:00p,@,Charlotte Hornets,L,,91,109,1,1,L 1
2,2018,ATL,2017-10-22,3:30p,@,Brooklyn Nets,L,,104,116,1,2,L 2
3,2018,ATL,2017-10-23,7:30p,@,Miami Heat,L,,93,104,1,3,L 3
4,2018,ATL,2017-10-26,8:00p,@,Chicago Bulls,L,,86,91,1,4,L 4


In [35]:
test['game_date'] = pd.to_datetime(test['game_date'])
test.head()

,id_season,game_nb,game_date,extdom,tm,opp,results,pts_tm,pts_opp,fg_tm,fga_tm,fg_prct_tm,3p_tm,3pa_tm,3p_prct_tm,ft_tm,fta_tm,ft_prct_tm,orb_tm,trb_tm,ast_tm,stl_tm,blk_tm,tov_tm,pf_tm,fg_opp,fga_opp,fg_prct_opp,3p_opp,3pa_opp,3p_prct_opp,ft_opp,fta_opp,ft_prct_opp,orb_opp,trb_opp,ast_opp,stl_opp,blk_opp,tov_opp,pf_opp
0,2018,1,2017-10-18,@,ATL,DAL,W,117,111,48,94,.511,9,18,.500,12,15,.800,14,50,20,11,6,13,18,38,86,.442,17,45,.378,18,21,.857,9,41,27,5,4,15,19
1,2018,2,2017-10-20,@,ATL,CHO,L,91,109,36,95,.379,7,30,.233,12,14,.857,6,38,19,11,4,13,29,37,86,.430,11,36,.306,24,30,.800,12,57,17,5,4,21,18
2,2018,3,2017-10-22,@,ATL,BRK,L,104,116,32,94,.340,7,27,.259,33,36,.917,16,48,21,6,2,16,25,41,87,.471,11,27,.407,23,34,.676,13,51,27,12,8,18,27
3,2018,4,2017-10-23,@,ATL,MIA,L,93,104,36,76,.474,7,17,.412,14,18,.778,6,41,14,9,1,20,17,39,85,.459,14,37,.378,12,18,.667,10,41,21,12,6,19,19
4,2018,5,2017-10-26,@,ATL,CHI,L,86,91,32,81,.395,11,28,.393,11,13,.846,7,40,22,10,5,10,23,31,86,.360,7,32,.219,22,29,.759,18,62,20,5,4,13,15


In [36]:
# join on game_date, id_season, extdom, tm 
final_test = pd.merge(
        test,
        test2[['id_season', 'tm', 'game_date', 'time_start', 'overtime', 'w_tot', 'l_tot', 'streak_w_l']],
        how='left',
        left_on=['id_season', 'tm', 'game_date'],
        right_on=['id_season', 'tm', 'game_date'])

final_test.head()

,id_season,game_nb,game_date,extdom,tm,opp,results,pts_tm,pts_opp,fg_tm,fga_tm,fg_prct_tm,3p_tm,3pa_tm,3p_prct_tm,ft_tm,fta_tm,ft_prct_tm,orb_tm,trb_tm,ast_tm,stl_tm,blk_tm,tov_tm,pf_tm,fg_opp,fga_opp,fg_prct_opp,3p_opp,3pa_opp,3p_prct_opp,ft_opp,fta_opp,ft_prct_opp,orb_opp,trb_opp,ast_opp,stl_opp,blk_opp,tov_opp,pf_opp,time_start,overtime,w_tot,l_tot,streak_w_l
0,2018,1,2017-10-18,@,ATL,DAL,W,117,111,48,94,.511,9,18,.500,12,15,.800,14,50,20,11,6,13,18,38,86,.442,17,45,.378,18,21,.857,9,41,27,5,4,15,19,8:30p,,1,0,W 1
1,2018,2,2017-10-20,@,ATL,CHO,L,91,109,36,95,.379,7,30,.233,12,14,.857,6,38,19,11,4,13,29,37,86,.430,11,36,.306,24,30,.800,12,57,17,5,4,21,18,7:00p,,1,1,L 1
2,2018,3,2017-10-22,@,ATL,BRK,L,104,116,32,94,.340,7,27,.259,33,36,.917,16,48,21,6,2,16,25,41,87,.471,11,27,.407,23,34,.676,13,51,27,12,8,18,27,3:30p,,1,2,L 2
3,2018,4,2017-10-23,@,ATL,MIA,L,93,104,36,76,.474,7,17,.412,14,18,.778,6,41,14,9,1,20,17,39,85,.459,14,37,.378,12,18,.667,10,41,21,12,6,19,19,7:30p,,1,3,L 3
4,2018,5,2017-10-26,@,ATL,CHI,L,86,91,32,81,.395,11,28,.393,11,13,.846,7,40,22,10,5,10,23,31,86,.360,7,32,.219,22,29,.759,18,62,20,5,4,13,15,8:00p,,1,4,L 4


In [37]:
final_test[final_test.isna().any(axis=1)].shape

(0, 46)

In [ ]:
def cleaned_and_combined(SCHEDULES_DF, GAMES_DF):

    #----------------------------------------------
    # SCHEDULES_DF - Re format date
    SCHEDULES_DF['game_date'] = pd.to_datetime(SCHEDULES_DF['game_date'])

    #----------------------------------------------
    # GAMES_DF - Recast date before merging the two dataset
    GAMES_DF['game_date'] = pd.to_datetime(GAMES_DF['game_date'])

    #----------------------------------------------
    # Join the two dataframes
    FinalDF = pd.merge(
        GAMES_DF,
        SCHEDULES_DF[['id_season', 'tm', 'game_date', 'time_start', 'overtime', 'w_tot', 'l_tot', 'streak_w_l']],
        how='left',
        left_on=['id_season', 'tm', 'game_date'],
        right_on=['id_season', 'tm', 'game_date'])

    return FinalDF



## NBA GAMES DATA - Single Year Team code

In [2]:
# URL to scrape
url = "https://www.basketball-reference.com/teams/BRK/2019/gamelog/"

# collect HTML data and create beautiful soup object:
# collect HTML data
html = urlopen(url)
        
# create beautiful soup object from HTML
soup = BeautifulSoup(html, "html.parser")

# use getText()to extract the headers into a list
titles = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]

rows = soup.findAll('tr')[2:]

rows_data = [[td.getText() for td in rows[i].findAll('td')]
                for i in range(len(rows))]


# create the dataframe
nba_finals = pd.DataFrame(rows_data)
cols = ["game_nb","game_date", 
        "extdom","opp","results",
        "pts_tm","pts_opp","fg_tm", 
        "fga_tm","fg_prct_tm", "3p_tm",
        "3pa_tm", "3p_prct_tm",
        "ft_tm","fta_tm","ft_prct_tm", 
        "orb_tm","trb_tm", "ast_tm",
        "stl_tm","blk_tm" ,"tov_tm",
        "pf_tm",
        "nc",
        "fg_opp",
        "fga_opp","fg_prct_opp","3p_opp",
        "3pa_opp","3p_prct_opp","ft_opp",
        "fta_opp","ft_prct_opp","orb_opp",
        "trb_opp","ast_opp", "stl_opp",
        "blk_opp","tov_opp", "pf_opp"]

nba_finals.columns =  cols
nba_finals.dropna()
nba_finals['id_season'] = 2019
nba_finals['tm'] = 'BKN'

nba_finals




,game_nb,game_date,extdom,opp,results,pts_tm,pts_opp,fg_tm,fga_tm,fg_prct_tm,3p_tm,3pa_tm,3p_prct_tm,ft_tm,fta_tm,ft_prct_tm,orb_tm,trb_tm,ast_tm,stl_tm,blk_tm,tov_tm,pf_tm,nc,fg_opp,fga_opp,fg_prct_opp,3p_opp,3pa_opp,3p_prct_opp,ft_opp,fta_opp,ft_prct_opp,orb_opp,trb_opp,ast_opp,stl_opp,blk_opp,tov_opp,pf_opp,id_season,tm
0,1,2018-10-17,@,DET,L,100,103,40,82,.488,5,27,.185,15,22,.682,5,39,28,9,5,17,23,,39,92,.424,6,24,.250,19,22,.864,14,46,21,5,5,14,20,2019,BKN
1,2,2018-10-19,,NYK,W,107,105,37,75,.493,12,30,.400,21,24,.875,12,55,22,2,6,22,23,,39,97,.402,9,28,.321,18,20,.900,12,36,14,10,1,3,19,2019,BKN
2,3,2018-10-20,@,IND,L,112,132,41,86,.477,16,37,.432,14,22,.636,9,40,25,6,6,20,18,,51,92,.554,16,24,.667,14,22,.636,10,40,21,12,5,10,18,2019,BKN
3,4,2018-10-24,@,CLE,W,102,86,38,87,.437,14,34,.412,12,15,.800,8,44,23,8,1,14,16,,34,89,.382,3,18,.167,15,16,.938,16,53,18,5,2,15,19,2019,BKN
4,5,2018-10-26,@,NOP,L,115,117,43,98,.439,19,40,.475,10,15,.667,16,54,26,6,8,18,22,,44,98,.449,9,23,.391,20,26,.769,15,46,23,11,11,11,20,2019,BKN
5,6,2018-10-28,,GSW,L,114,120,42,87,.483,20,42,.476,10,15,.667,5,37,28,5,3,15,20,,44,89,.494,11,33,.333,21,24,.875,12,47,28,11,7,12,16,2019,BKN
6,7,2018-10-29,@,NYK,L,96,115,34,84,.405,11,38,.289,17,22,.773,11,32,22,6,7,10,20,,45,92,.489,9,24,.375,16,18,.889,16,53,21,5,3,13,21,2019,BKN
7,8,2018-10-31,,DET,W,120,119,43,101,.426,12,38,.316,22,30,.733,18,55,26,5,3,16,33,,39,93,.419,11,34,.324,30,39,.769,14,55,16,8,6,16,30,2019,BKN
8,9,2018-11-02,,HOU,L,111,119,42,85,.494,15,34,.441,12,20,.600,9,38,22,5,4,10,28,,41,83,.494,15,39,.385,22,32,.688,11,41,22,4,10,9,21,2019,BKN
9,10,2018-11-04,,PHI,W,122,97,50,105,.476,9,28,.321,13,18,.722,15,38,27,11,3,9,28,,31,65,.477,4,20,.200,31,41,.756,9,46,16,6,1,27,21,2019,BKN


## NBA GAMES OVERTIME AND SCHEDULE - Single Year Team code 

In [23]:
from urllib.error import HTTPError, URLError

# URL to scrape
url = "https://www.basketball-reference.com/teams/CHA/2019_games.html"



if str(requests.get(url)) != '<Response [404]>':
# requests.get(url).raise_for_status()

# try:
#     response = requests.get(url)
#     response.raise_for_status()
# except HTTPError as hp:
#     print('not working')
# else:
#     print("it's worked")

'<Response [404]>'

In [6]:
from urllib.error import HTTPError
import requests
# URL to scrape
url = "https://www.basketball-reference.com/teams/MIA/2000_games.html"


# collect HTML data and create beautiful soup object:
# collect HTML data
html = urlopen(url)
        
# create beautiful soup object from HTML
soup = BeautifulSoup(html)

# use getText()to extract the headers into a list
titles = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]

rows = soup.findAll('tr')[2:]

rows_data = [[td.getText() for td in rows[i].findAll('td')]
                for i in range(len(rows))]


# create the dataframe
nba_finals = pd.DataFrame(rows_data)
cols = ['game_date', 'time_Start', 'nc1', 'nc2', 'extdom', 'opponent', 'w_l', 'overtime', 'pts_tm', 'pts_opp', 'W', 'L', 'streak_w_l', 'nc3']
nba_finals.columns =  cols
# nba_finals = nba_finals.dropna()
# nba_finals['id_season'] = 2019
# nba_finals['tm'] = 'BKN'

nba_finals




ValueError: Length mismatch: Expected axis has 13 elements, new values have 14 elements

In [59]:
import requests
def webscrappe_nba_schedule_overtime_data(SEASON):
    
    #------------------------------------------------------
    # Get team reference data 

    team_city_refdata = pd.read_csv('./constants/team_city_refdata.csv', sep = ';')

    #------------------------------------------------------
    # Initialization of the dataframe to fill-in
    schedules = pd.DataFrame()

    #------------------------------------------------------
    # For Loop Throught all the team abrev for the given season

    for index, row in team_city_refdata.iterrows():
        
        # URL to scrape
        team = row['team_abrev']

        # URL to scrape
        url = f"https://www.basketball-reference.com/teams/{team}/{SEASON}_games.html"

        if str(requests.get(url)) != '<Response [404]>':

            # collect HTML data and create beautiful soup object:
            # collect HTML data
            html = urlopen(url)
                    
            # create beautiful soup object from HTML
            soup = BeautifulSoup(html)

            # use getText()to extract the headers into a list
            titles = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]

            rows = soup.findAll('tr')[2:]

            rows_data = [[td.getText() for td in rows[i].findAll('td')]
                            for i in range(len(rows))]

            if len(rows_data) != 0:
                # create the dataframe
                schedule_tmp = pd.DataFrame(rows_data)

                cols = ['game_date', 'time_Start', 'nc1', 'nc2', 'extdom', 'opponent', 'w_l', 'overtime', 'pts_tm', 'pts_opp', 'W', 'L', 'streak_w_l', 'nc3']
                schedule_tmp.columns =  cols

                schedule_tmp = schedule_tmp.dropna()

                schedule_tmp['id_season'] = SEASON
                schedule_tmp['tm'] = team

                schedules = schedules.append(schedule_tmp)
        
    return schedules

In [60]:
test = webscrappe_nba_schedule_overtime_data(2019)


,game_date,time_Start,nc1,nc2,extdom,opponent,w_l,overtime,pts_tm,pts_opp,W,L,streak_w_l,nc3,id_season,tm
0,"Fri, Oct 19, 2018",8:00p,,Box Score,@,Memphis Grizzlies,L,,117,131,0,2,L 2,,2019,ATL
1,"Sun, Oct 21, 2018",6:00p,,Box Score,@,Cleveland Cavaliers,W,,133,111,1,2,W 1,,2019,ATL
2,"Wed, Oct 24, 2018",7:00p,,Box Score,,Dallas Mavericks,W,,111,104,2,2,W 2,,2019,ATL
3,"Sat, Oct 27, 2018",7:30p,,Box Score,,Chicago Bulls,L,,85,97,2,3,L 1,,2019,ATL
4,"Mon, Oct 29, 2018",7:00p,,Box Score,@,Philadelphia 76ers,L,,92,113,2,4,L 2,,2019,ATL
5,"Tue, Oct 30, 2018",7:00p,,Box Score,@,Cleveland Cavaliers,L,,114,136,2,5,L 3,,2019,ATL
6,"Thu, Nov 1, 2018",7:30p,,Box Score,,Sacramento Kings,L,,115,146,2,6,L 4,,2019,ATL
7,"Sat, Nov 3, 2018",7:30p,,Box Score,,Miami Heat,W,,123,118,3,6,W 1,,2019,ATL
8,"Tue, Nov 6, 2018",7:00p,,Box Score,@,Charlotte Hornets,L,,102,113,3,7,L 1,,2019,ATL
9,"Wed, Nov 7, 2018",7:30p,,Box Score,,New York Knicks,L,,107,112,3,8,L 2,,2019,ATL


In [50]:
test['tm'][test['game_nb']<83].value_counts()

KeyError: 'tm'